**Charger les librairies nécéssaires**

In [19]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

**Chargement des données**

In [20]:
# Lecture du fichier nettoyé
df = pd.read_csv("data/avito_cars_clean.csv")

# Afficher les cinq premières lignes des données
df.head()

,annee,boite,carburant,kilometrage,marque,modele,nombre_portres,premiere_main,puissance_fiscale,etat,prix
0,2023,Automatique,Essence,22499.5,Alfa Romeo,Tonale,5,Oui,8,Excellent,306000
1,2018,Manuelle,Diesel,104999.5,Dacia,Duster,5,Oui,6,Excellent,155000
2,2023,Automatique,Diesel,67499.5,Hyundai,Accent,5,Oui,6,Excellent,179000
3,2023,Automatique,Diesel,22499.5,Peugeot,5008,5,Oui,8,Excellent,349000
4,2021,Automatique,Diesel,77499.5,Kia,Sportage,5,Oui,12,Très bon,685000


**les colonnes disponibles**

In [21]:
df.columns

Index(['annee', 'boite', 'carburant', 'kilometrage', 'marque', 'modele',
       'nombre_portres', 'premiere_main', 'puissance_fiscale', 'etat', 'prix'],
      dtype='object')

**Voir s'il y a des données manquantes**

In [22]:
df.isnull().sum()


annee                0
boite                0
carburant            0
kilometrage          0
marque               0
modele               0
nombre_portres       0
premiere_main        0
puissance_fiscale    0
etat                 0
prix                 0
dtype: int64

**Sélectionner les colonnes ***utiles*****

In [23]:
# Sélectionner les nouvelles variables
features = ['annee', 'kilometrage', 'puissance_fiscale', 'etat', 'marque', 'modele']
target = 'prix'

# Créer X (features) et y (target)
X = df[features]
y = df[target]

# One-Hot Encoding des variables catégorielles (etat, marque, modele)
X = pd.get_dummies(X, drop_first=True)

# Supprimer les lignes avec des valeurs manquantes
X = X.dropna()
y = y[X.index]  # Assurer la correspondance avec X

# Séparer en train / test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Création et entraînement de modèle**

In [24]:
# Créer le modèle
model = LinearRegression()

# Entraîner le modèle
model.fit(X_train, y_train)


LinearRegression()

In [27]:
from sklearn.preprocessing import StandardScaler

# Apply before model fitting
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Use scaled data for model training and predictions
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

# Remove extreme outliers in price
Q1 = df['prix'].quantile(0.25)
Q3 = df['prix'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df[(df['prix'] >= lower_bound) & (df['prix'] <= upper_bound)]

**Evaluation du modèle :**
on calcule les métriques pour évaluer la performance

In [28]:
# Prédictions sur les données de test
y_pred = model.predict(X_test)

# Calculer les métriques d'évaluation
mae = mean_absolute_error(y_test, y_pred)
rmse = (mean_squared_error(y_test, y_pred))**0.5
r2 = r2_score(y_test, y_pred)

# Approximation de l'accuracy (pas une véritable accuracy)
accuracy_approx = 100 - rmse

# 8. Afficher les résultats
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² (coefficient de détermination): {r2}")
print(f"Approximation de l'Accuracy: {accuracy_approx}%")

Mean Absolute Error (MAE): 8.149218729896494e+16
Root Mean Squared Error (RMSE): 1.113328445427156e+17
R² (coefficient de détermination): -9.876059533876221e+23
Approximation de l'Accuracy: -1.113328445427155e+17%


c:\project\venv\Lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  def _repr_html_inner(self):
